In [1]:
import pandas as pd

results = pd.read_feather('../Figure1/saved_data_plot/results_benchmark_algos.feather').set_index('reference')
results = results[results['dataset']!= 'lncRNA']
results.loc[results['dataset']=='viral_fragments', 'dataset'] = 'Viral mRNA'
results.loc[results['dataset']=='lncRNA_nonFiltered', 'dataset'] = 'Long ncRNA'
results.loc[results['dataset']=='archiveII', 'dataset'] = 'ArchiveII'
results.loc[results['model']=='RNAstructure', 'model'] = 'RNAstructure Fold'

In [2]:
# Group the data by model and dataset and calculate the mean for each group
grouped = results.groupby(['model', 'dataset']).mean(numeric_only=True).reset_index()

# Pivot the table to create a multi-level column structure
pivot_df = pd.pivot_table(grouped, index='model', columns='dataset', values=['Precision', 'Recall', 'F1'])

# Swap the level of the columns to have dataset as the top level and the metrics as the second level
pivot_df = pivot_df.swaplevel(i=0, j=1, axis=1).sort_index(axis=1)

# Define the new order for the models and reorder the rows
new_order = ['RNAstructure Fold', 'EternaFold', 'MXFold2', 'UFold']
pivot_df = pivot_df.reindex(new_order)

pivot_df = pivot_df.reindex(columns=pivot_df.columns.reindex(['Precision', 'Recall', 'F1'], level=1)[0])[['PDB', 'ArchiveII', 'Viral mRNA', 'Long ncRNA']]

pivot_df = pivot_df.style\
            .format(precision=3)\
            .highlight_max(axis=0, props="font-weight:bold;font-color:black;")\
            .background_gradient(axis=1, vmin=-0.1, vmax=1, cmap="viridis", text_color_threshold=0)\
            .set_properties(**{'text-align': 'center'})\
            .set_table_styles(
                        [{"selector": "th", "props": [('text-align', 'center')]},
                        # {"selector": "tbody td", "props": [("border", "1px solid")]},
                        # {"selector": "th", "props": [("border", "1px solid")]}
                        ])
pivot_df


In [4]:
pivot_df.to_excel('tables/T1_algo_benchmark.xlsx')